In [ ]:
import os
import random
from PIL import Image
import cv2
import numpy as np
WIDTH = 707
HEIGHT = 557

In [ ]:
icon_files = os.listdir('icons/')
icons = {}
for f in icon_files:
    icons[f[:-4]] = Image.open(f'icons/{f}')
icons = list(icons.items())
background = Image.open('background.jpg')
background = background.convert('RGBA')

In [ ]:
QUADRANTS = [
    (0, 0),
    (1, 0),
    (2, 0),
    (0, 1),
    (1, 1),
    (2, 1),
    (0, 2),
    (1, 2),
    (2, 2),
]

In [ ]:
def remove_background(image):
    image = np.array(image)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    l_h = 0
    u_h = 179
    l_s = 0
    u_s = 60
    l_v = 0
    u_v = 255

    lower_green = np.array([l_h, l_s, l_v])
    upper_green = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, lower_green, upper_green)
    mask_inv = cv2.bitwise_not(mask)

    fg = cv2.bitwise_and(image, image, mask=mask_inv)
    return Image.fromarray(fg)

In [ ]:
DIRECTORY = 'images_224/'
random.seed(5)
!rm -rf $DIRECTORY
!mkdir -p $DIRECTORY

for i in range(10000):
    used_icons = []
    image = background.copy()
    quadrants = list(QUADRANTS)
    usable_icons = list(icons)
    
    for _ in range(random.randint(1, 5)):
    #for _ in range(0, 1):
        icon_name, icon = random.choice(usable_icons)
        usable_icons.remove((icon_name, icon))
        used_icons.append(icon_name)

        # resize
        width = random.randint(200, 500)
        height = int(HEIGHT * (width / WIDTH))
        icon = icon.resize((width, height))

        # rotate
        icon = icon.rotate(random.randint(0, 360))

        # position
        q = random.choice(quadrants)
        quadrants.remove(q)
        x = q[0] * 250 # + random.randint(0, 100)
        y = q[1] * 250 # + random.randint(0, 100)

        image.paste(icon, (x, y), icon)
    image = image.convert('RGB')
    # image = remove_background(image)
    image = image.resize((224, 224), Image.LANCZOS)
    filename = '_'.join(used_icons)
    image.save(f'{DIRECTORY}/{i:05d}_{filename}.png')